# Calculating Zonal Statistics on Rasters
So far we have chained together two geospatial operations, reprojection and raster math, to produce our Canopy Height Model (CHM) and learned how to read vector data with `geopandas`.

In [3]:
from xrspatial import zonal_stats
import xarray as xr
import geopandas as gpd
import rasterio
import rioxarray

# reads the surface and terrain
surface_HARV = rioxarray.open_rasterio("data/NEON-GEO-PYTHON-DATASETS/NEON-DS-Airborne-Remote-Sensing/HARV/DSM/HARV_dsmCrop.tif", masked=True)
terrain_HARV_UTM18 = rioxarray.open_rasterio("data/NEON-GEO-PYTHON-DATASETS/NEON-DS-Airborne-Remote-Sensing/HARV/DTM/HARV_dtmCrop_UTM18.tif", masked=True)

# reprojects terrain
terrain_HARV_matched = terrain_HARV_UTM18.rio.reproject_match(surface_HARV)

# creates CHM
canopy_HARV = surface_HARV - terrain_HARV_matched

# reads roads
roads = gpd.read_file("data/NEON-GEO-PYTHON-DATASETS/NEON-DS-Site-Layout-Files/HARV/HARV_roads.shp")

Now that we have our CHM, we can compute statistics with it to understand how canopy height varies across our study area. If we want to focus on specific areas or zones of interest, we can use zonal statistics. To do this, we'll import the `zonal_stats` function from the package `xrspatial`.

## Zonal Statistics with xarray-spatial
We often want to perform calculations for specific zones in a raster. These can be delineated by points, lines, or polygons. In the case of HARV, we hzve a shapefile that contains lines representing walkways, footpaths, and roads. A single function call, `xrspatial.zonal_stats` can calculate the minimum, maximum, mean, median, and standard deviation for each line zone in our CHM.

In order to accomplish this, we first need to rasterize our `roads` geodataframe with the `rasterio.features.rasterize` function. This will produce a grid with number values representing each type of line, with numbers varying with the type of line. This grid's values will then represent each of our zones for the `xrspatial.zonal_stats` function, where each pixel in the zone grid overlaps with a corresponding pixel in our CHM raster.

Before rasterizing, we need to do a little work to make a variable, `shapes`, that associates a line with a unique number to represent that line. This variable will later be used as the first argument to `rasterio.features.rasterize`.